## High Level Guide to This Project

1. Web Scraping and Getting the Data
    
    a. Scrape the Home Chef website to get all the menu items and categories from current and future weeks
    
    b. Manipulate an existing restaurant ratings dataset to generate some "user data". 

2. Making Recommendations

    a. Build the Recommendation Engine (Cross-Collaborative Filter). Tune Recommendations to minimize RMSE.
    
    b. Make Predictions using the Engine



## Part 1a: Web Scraping

I need to generate a matrix of customers and menu items before performing the recommendation machine learning. For customers and ratings, I will generate some randomized data. For the menu items, I'll scrape them from the Home Chef Website for practice.

In [1]:
from bs4 import BeautifulSoup
import requests

url = "https://www.homechef.com/our-menu"
r = requests.get(url)
html_doc = r.text
soup = BeautifulSoup(html_doc, 'html.parser')

In [2]:
#let's get links to all the other weeks of menu items available on the site
import re
weeks = [url]

for link in soup.find_all('a'):
    href = link.get('href')
    if re.match('/our-menus/', href) and not re.search('standard$', href):
        weeks.append('https://www.homechef.com' + href)
print(weeks)

['https://www.homechef.com/our-menu', 'https://www.homechef.com/our-menus/05-feb-2018', 'https://www.homechef.com/our-menus/12-feb-2018', 'https://www.homechef.com/our-menus/19-feb-2018', 'https://www.homechef.com/our-menus/26-feb-2018']


In [3]:
import time
meal_categs = {}

for url in weeks:
    time.sleep(1)
    
    r = requests.get(url)
    html_doc = r.text
    soup = BeautifulSoup(html_doc, 'html.parser')

    cards = soup.find_all(id='meal')
    for card in cards:
        values = []
        meal = card.h2.text
    
        categs = card.find_all('li')
        for categ in categs:
            values.append(categ.span.text)
    
        icons = card.find_all('i')
        for icon in icons:
            values.append(icon['data-tooltip'])
    
        meal_categs[meal] = values
    
print(meal_categs)

{'Barbacoa Steak Tacos': ['Milk', 'Wheat', 'Soy'], 'Roasted Salmon with Ginger-Scallion Sauce': ['Fish', 'Wheat', 'Soy', 'Calorie-Conscious', 'Carb-Conscious'], 'Classic Chicken Piccata': ['Milk', 'Wheat'], 'Buffalo Chicken Dip': ['Milk', 'Eggs', 'Wheat', 'Soy'], 'Caprese Burger': ['Milk', 'Eggs', 'Wheat'], 'Chicken with Lemon-Chive Crema': ['Milk', 'Calorie-Conscious', 'Carb-Conscious'], 'Spicy Cajun Shrimp and White Cheddar Grits': ['Milk', 'Peanuts', 'Shellfish', 'Calorie-Conscious'], 'Pork Chop with Parsley-Garlic Butter': ['Milk', 'Calorie-Conscious', 'Carb-Conscious'], 'Taco Salad Pizza': ['Milk', 'Eggs', 'Wheat', 'Soy', 'Vegetarian'], 'Ponzu Noodles and Charred Green Beans': ['Wheat', 'Soy', 'Tree Nuts', 'Calorie-Conscious', 'Vegetarian'], 'Creamy Tomato Soup': ['Milk', 'Wheat', 'Soy', 'Calorie-Conscious', 'Vegetarian'], 'Apple Cherry Chicken Salad': ['Tree Nuts', 'Calorie-Conscious', 'Carb-Conscious'], 'Mojito Mint Smoothie': ['Milk', 'Vegetarian'], 'Seasonal Fruit Basket': ['V

In [4]:
#Create a set of categories from the dictionary values
categories = set()
for i in list(meal_categs.values()):
    for j in i:
        categories.add(j)
print(categories)
print(len(categories))

{'Milk', 'Eggs', 'Peanuts', 'Shellfish', 'Wheat', 'Vegetarian', 'Fish', 'Carb-Conscious', 'Tree Nuts', 'Calorie-Conscious', 'Soy'}
11


In [5]:
#Create a set of menu items from the dictionary keys
meals = set(meal_categs.keys())
print(list(meals)[:5])
print(len(meals))

['Brussels & Brown Butter Risotto', 'Strawberry Orange Smoothie', 'Salmon with Dill Crema', 'Cajun Tilapia with Jambalaya Pilaf', 'Ponzu Noodles and Charred Green Beans']
66


In [6]:
#Next we need to make a table that shows what categories
#the menu item falls into.
import pandas as pd
import numpy as np

m = len(meals)
n = len(categories)

df_menu = pd.DataFrame(data=np.zeros((m,n)), columns=categories, index=meals)
df_menu.head()

,Milk,Eggs,Peanuts,Shellfish,Wheat,Vegetarian,Fish,Carb-Conscious,Tree Nuts,Calorie-Conscious,Soy
Brussels & Brown Butter Risotto,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Strawberry Orange Smoothie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Salmon with Dill Crema,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cajun Tilapia with Jambalaya Pilaf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ponzu Noodles and Charred Green Beans,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
#Now let's populate the table based on the dictionary

for meal in meals:
    for categ in meal_categs[meal]:
        df_menu.ix[meal][categ] = 1
df_menu.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """


,Milk,Eggs,Peanuts,Shellfish,Wheat,Vegetarian,Fish,Carb-Conscious,Tree Nuts,Calorie-Conscious,Soy
Brussels & Brown Butter Risotto,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
Strawberry Orange Smoothie,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
Salmon with Dill Crema,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
Cajun Tilapia with Jambalaya Pilaf,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
Ponzu Noodles and Charred Green Beans,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0


The table above could be used for Content-based filtering. However, for this exercise I'll be using Collaborative filtering (User-based). If I were to create a method for content-based filtering, I would want to create some more descriptive columns for each dish. These additional columns could be based on ingredients, region, or cooking methods.

## Part 1b: Generating Random User Meal Ratings Data

Since I don't have access to the actual Home Chef Ratings, I found another ratings data source online. This dataset hold ratings for restaurants, so somewhat similar to what we are looking for. Since the ratings are broken out into three categories, I will do some quick algebra to convert to single rating on a 5-star scale (Home Chef rating scale).

In [8]:
#we need a new table that will hold user_ids and reviews of menu items
d = pd.read_csv('RCData/rating_final.csv')
d['r'] = np.round((d.rating + d.food_rating + d.service_rating)*5/6)
d.head()

,userID,placeID,rating,food_rating,service_rating,r
0,U1077,135085,2,2,2,5.0
1,U1077,135038,2,2,1,4.0
2,U1077,132825,2,2,2,5.0
3,U1077,135060,1,2,2,4.0
4,U1068,135104,1,1,2,3.0


Checking the distribution of reviews. Looks ok.

In [9]:
d.r.value_counts().sort_index()

0.0    193
1.0     43
2.0    312
3.0    178
4.0    142
5.0    293
Name: r, dtype: int64

I'll just grab the restaurants with the most reviews up to the number of meals on the available Home Chef menus (62 at the time I did this).

In [10]:
top_places = d.groupby('placeID').size().sort_values(ascending=False).index[:len(meals)]

Next I'll take just those top restaurants and the users that reviewed them. I'll put them into a pandas dataframe as below.

In [11]:
df_ratings = d[d.placeID.isin(top_places)]
df_ratings = df_ratings.pivot_table(values='r', columns='placeID', index='userID')
df_ratings.columns = meals
df_ratings.head()

,Brussels & Brown Butter Risotto,Strawberry Orange Smoothie,Salmon with Dill Crema,Cajun Tilapia with Jambalaya Pilaf,Ponzu Noodles and Charred Green Beans,Cashew Tofu,Chipotle Ranch Chicken Salad,Frutti Tutti Smoothie,Taco Salad Pizza,Buffalo Chicken Dip,...,Mexican Street Corn Flatbread,Lasagna Arrabiata,Shrimp Pomodoro,Wild Mushroom and Pesto Linguine,Mesquite Filet Mignon,Chicken with Mushroom Gravy,Shrimp Lo Mein,Mediterranean Couscous Bowl,Chile Rellenos Rice Casserole,Classic Chicken Piccata
userID,,,,,,,,,,,,,,,,,,,,,
U1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
U1002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,2.0,NaN
U1003,NaN,4.0,NaN,NaN,NaN,5.0,NaN,4.0,NaN,NaN,...,5.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U1004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN
U1005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,...,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Part 2a: Cross Collaborative Filtering, Tuning the Recommendation Engine

Now with the data ready, it's time to perform the recommendation prediction. There are several Python packages for doing this, but I'll build some functions by hand to do the work for the purposes of this project.

First I'll melt (unpivot) the data and get rid of NaN values to make it easier to manipulate

In [12]:
df = df_ratings.reset_index()
df = pd.melt(df, id_vars=['userID'], var_name=['meal'])
df = df[df.value.notnull()]
df.rename(columns={'value': 'rating'}, inplace=True)
df.head()

,userID,meal,rating
5,U1006,Brussels & Brown Butter Risotto,2.0
6,U1007,Brussels & Brown Butter Risotto,2.0
11,U1013,Brussels & Brown Butter Risotto,2.0
29,U1033,Brussels & Brown Butter Risotto,1.0
40,U1046,Brussels & Brown Butter Risotto,2.0


Here's a helper function to split the data into training and test sets.

In [13]:
def assign_to_set(df):
    sampled_ids = np.random.choice(df.index,
                                   size=np.int64(np.ceil(df.index.size * 0.2)),
                                   replace=False)
    df.ix[sampled_ids, 'for_testing'] = True
    return df

df['for_testing'] = False
grouped = df.groupby('userID', group_keys=False).apply(assign_to_set)
df_train = df[grouped.for_testing == False]
df_test = df[grouped.for_testing == True]
print (df.shape)
print (df_train.shape)
print (df_test.shape)
assert (len(df_train.index & df_test.index) == 0)

(862, 4)
(636, 4)
(226, 4)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """


We'll also need evaluation metrics:

In [14]:
def compute_rmse(y_pred, y_true):
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))

def evaluate(estimate_f):
    ids_to_estimate = zip(df_test.userID, df_test.meal)
    estimated = np.array([estimate_f(u,m) for (u,m) in ids_to_estimate])
    real = df_test.rating.values
    return compute_rmse(estimated, real)

I'll check the evaluation function by just guessing 3 for every meal.

In [15]:
def my_estimate_function(userID, meal):
    return 3

In [16]:
print ('RMSE for my estimate function: %s' % evaluate(my_estimate_function))

RMSE for my estimate function: 1.73077301263


Next I'll create some different similarity functions to try:

In [17]:
def euclidean(s1, s2):
    diff = s1 - s2
    return 1 / (1 + np.sqrt(np.sum(diff ** 2)))

def pearson(s1, s2):
    s1_c = s1 - s1.mean()
    s2_c = s2 - s2.mean()
    return np.sum(s1_c * s2_c) / np.sqrt(np.sum(s1_c ** 2) * np.sum(s2_c ** 2))

Next, create a Class for the collaborative filtering recommender:

In [18]:
class CollabPearsonReco:

    def learn(self):        
        self.all_user_profiles = df.pivot_table('rating', index='meal', columns='userID')

    def estimate(self, userID, meal):     
        user_condition = df_train.userID != userID
        meal_condition = df_train.meal == meal
        ratings_by_others = df_train.loc[user_condition & meal_condition]
        if ratings_by_others.empty: 
            return 3.0
        
        ratings_by_others.set_index('userID', inplace=True)
        their_ids = ratings_by_others.index
        their_ratings = ratings_by_others.rating
        their_profiles = self.all_user_profiles[their_ids]
        user_profile = self.all_user_profiles[userID]
        sims = their_profiles.apply(lambda profile: pearson(profile, user_profile), axis=0)
        ratings_sims = pd.DataFrame({'sim': sims, 'rating': their_ratings})
        ratings_sims = ratings_sims[ratings_sims.sim > 0]
        if ratings_sims.empty:
            return their_ratings.mean()
        else:
            return np.average(ratings_sims.rating, weights=ratings_sims.sim)
        
reco = CollabPearsonReco()
reco.learn()
print ('RMSE for CollabPearsonReco: %s' % evaluate(reco.estimate))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


RMSE for CollabPearsonReco: 1.63715778644


In [19]:
class CollabEuclidReco:
    
    def learn(self):
        self.all_user_profiles = df.pivot_table('rating', index='meal', columns='userID')
        
    def estimate(self, userID, meal):
        user_condition = df_train.userID != userID
        meal_condition = df_train.meal == meal
        ratings_by_others = df_train.loc[user_condition & meal_condition]
        if ratings_by_others.empty: 
            return 3.0
        
        ratings_by_others.set_index('userID', inplace=True)
        their_ids = ratings_by_others.index
        their_ratings = ratings_by_others.rating
        their_profiles = self.all_user_profiles[their_ids]
        user_profile = self.all_user_profiles[userID]
        sims = their_profiles.apply(lambda profile: euclidean(profile, user_profile), axis=0)
        ratings_sims = pd.DataFrame({'sim': sims, 'rating': their_ratings})
        ratings_sims = ratings_sims[ratings_sims.sim > 0]
        if ratings_sims.empty:
            return their_ratings.mean()
        else:
            return np.average(ratings_sims.rating, weights=ratings_sims.sim)
    
    def recommend(self, userID, n=5):        
        other_ratings = df[df.userID != userID]
        ratings_by_others.set_index('userID', inplace=True)
        their_ids = ratings_by_others.index
        sims = their_profiles.apply(lambda profile: euclidean(profile, user_profile), axis=0)
        
reco = CollabEuclidReco()
reco.learn()
print ('RMSE for CollabPearsonReco: %s' % evaluate(reco.estimate))

RMSE for CollabPearsonReco: 1.46098599725


Euclidean similarity had a better RMSE than Pearson similartiy (1.46 vs 1.64 respectively), so I'll use the Euclidean function going forward. With a larger dataset and preferably real empirical data, I would tune the algorithm to get the lowest RMSE.

## Part 2b: Using the Recommendation Engine to Make Predictions

Next, I'll look at a random user to see how well the model can predict something the user has already rated. In this case, we will look at the prediction for the "Tex-Mex Steak Sandwich".

In [20]:
df[df.userID=='U1025']

,userID,meal,rating,for_testing
891,U1025,Frutti Tutti Smoothie,0.0,False
1015,U1025,Taco Salad Pizza,5.0,False
3247,U1025,Tex-Mex Steak Sandwich,2.0,False
4487,U1025,Fontina and Caramelized Onion Flatbread,2.0,False
6099,U1025,Strawberry Fields Smoothie,5.0,False
6719,U1025,Barbacoa Steak Tacos,5.0,False


In [22]:
print("Estimated Rating: ", reco.estimate(userID='U1025',meal='Tex-Mex Steak Sandwich'))
print("Actual Rating: ", df[(df.userID=='U1025') & (df.meal=='Tex-Mex Steak Sandwich')].rating.item())

Estimated Rating:  2.85621667915
Actual Rating:  2.0


Pretty good. You can see for this case, the model was able to predict the user's actual rating fairly closely. Although this is only one case of many, we know we are on the right track.

Now I'll take a user and provide the top 5 recommendations for the next meal she should try. For this case, I'm going to convert the dataframe into a dictionary for easier manipulation.

In [23]:
from pandas import compat

rating_dict = df.pivot_table('rating', index='meal', columns='userID',)
def to_dict_dropna(data):
  return dict((k, v.dropna().to_dict()) for k, v in compat.iteritems(data))
rating_dict = to_dict_dropna(rating_dict)
rating_dict

{'U1001': {'Chicken Diane': 2.0,
  'Chicken with Beurre Blanc': 3.0,
  'Chicken with Mushroom Gravy': 2.0,
  'Frutti Tutti Smoothie': 4.0,
  "N'awlins Shrimp Po'Boy": 5.0,
  'Taco Salad Pizza': 2.0,
  'Turkey and Avocado Tostadas': 2.0},
 'U1002': {'Chicken Milanesa': 2.0,
  'Chicken with Mushroom Gravy': 2.0,
  'Chile Rellenos Rice Casserole': 2.0,
  'Coffee-Rubbed Steak ': 3.0,
  'Creamy Tomato Soup': 3.0,
  'Frutti Tutti Smoothie': 3.0,
  'Hearty Steak and Red Wine Stew': 5.0,
  'Roasted Salmon with Ginger-Scallion Sauce': 3.0,
  'Sweet Potato and Black Bean Taco Bowl': 2.0},
 'U1003': {'Cashew Tofu': 5.0,
  'Frutti Tutti Smoothie': 4.0,
  'Hearty Steak and Red Wine Stew': 4.0,
  'Mexican Street Corn Flatbread': 5.0,
  'Roasted Salmon with Ginger-Scallion Sauce': 5.0,
  'Shrimp Pomodoro': 5.0,
  'Strawberry Fields Smoothie ': 0.0,
  'Strawberry Orange Smoothie': 4.0,
  'Sweet Potato and Black Bean Taco Bowl': 2.0},
 'U1004': {'Chicken with Lemon-Chive Crema': 5.0,
  'Chile Rellenos 

I also need to rewrite the Euclidean distance function for the dictionary data structure

In [24]:
from math import sqrt

def sim_distance(prefs, person1, person2):
    si={}
    for item in prefs[person1]:
        if item in prefs[person2]:
            si[item] = 1
            
    if len(si) == 0: return 0
    sum_of_squares = sum([pow(prefs[person1][item]-prefs[person2][item],2) for item in si])
    return 1/(1+sqrt(sum_of_squares))

Now I'll write a function to get a ranked list of recommendations.

In [25]:
# Gets recommendations for a person by using a weighted average
# of every other user's rankings
def getRecommendations(prefs,userID,similarity=sim_distance, n=5):
    totals={}
    simSums={}
    for other in prefs:
        # don't compare me to myself
        if other==userID: continue
        sim=similarity(prefs,userID,other)

        # ignore scores of zero or lower
        if sim <= 0: continue
        for item in prefs[other]:

            # only score movies I haven't seen yet
            if item not in prefs[userID] or prefs[userID][item]==0:
                # Similarity * Score
                totals.setdefault(item,0)
                totals[item] += prefs[other][item]*sim
                # Sum of similarities
                simSums.setdefault(item,0)
                simSums[item]+=sim

    # Create the normalized list
    rankings=[(total/simSums[item],item) for item,total in totals.items(  )]

    # Return the sorted list
    rankings.sort()
    rankings.reverse()
    return rankings[:n]

In [26]:
getRecommendations(rating_dict,'U1025', n=10)

[(5.0, 'Manhattan Cod and Tomato Stew'),
 (4.2316027367034543, "N'awlins Shrimp Po'Boy"),
 (3.942810114744542, 'Mexican Street Corn Flatbread'),
 (3.8095238095238098, 'Pork Tenderloin with Roasted Pears'),
 (3.5625, 'Huli Huli Chicken Rice Bowl'),
 (3.5263157894736841, 'Seasonal Fruit Basket'),
 (3.4630193016579667, 'Wild Mushroom and Pesto Linguine'),
 (3.3250691052501495, 'Farmhouse Fried Chicken'),
 (3.318956467231482, 'Lasagna Arrabiata'),
 (3.2899375914569067, 'Pecan-Crusted Chicken')]

And now we have a way to suggest the user which meals they might like to try next. The higher the score, more likely we believe they will rate the meal highly. Since the data came from another dataset, the meals don't necessarily make sense with what the user previously rated. However, with the actual user data, we would be able to predict preferences that would make more sense.